## Simple Mission

This notebook serves to check the ardupilot installation

In [1]:
import os
import subprocess
import time

from config import ARDUPILOT_VEHICLE_PATH, LOGS_PATH, VEH_PARAMS_PATH
from helpers import clean
from mavlink.util import connect
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agrre with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid 1 "
    f"--no-rebuild "
    f"--use-dir={LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f'--custom-location={spawn_str}'
)


# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal","--title=planner","--", 
                  "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--title=planner', '--', '...>

## 1. Connect to the vehicle

In [3]:
time.sleep(1)
conn = connect("tcp:127.0.0.1:5760")
conn.wait_heartbeat() 
print("✅ Heartbeat received")


✅ Heartbeat received


## Create auto plan from mission file

The mission is loaded from "plan/missions/simple_mission.waypoints"

In [4]:
plan = Plan.auto(mission_name="simple_mission",start_delay=5)
plan

🕓 <Plan '📋 '>
  🕓 <Action '📤 UPLOAD_MISSION'>
    🕓 <Step '🔹 clear uav missions'>
    🕓 <Step '🔹 load mission simple_mission'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️ MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '⏳ WAIT'>
    🕓 <Step '🔹 wait'>
  🕓 <Action '🚀 START_MISSION'>
    🕓 <Step '🔹 start_mission'>
  🕓 <Action '📤 UPLOAD_MISSION'>
    🕓 <Step '🔹 monitoring'>

# Execute Auto Plan

In [5]:
plan.bind(conn,verbose=3)
while plan.state != State.DONE:
    plan.act()

Vehicle 1: Step 'clear uav missions' is now connected ✅🔗
Vehicle 1: Step 'load mission simple_mission' is now connected ✅🔗
Vehicle 1: Action 'UPLOAD_MISSION' is now connected ✅🔗
Vehicle 1: Action 'UPLOAD_MISSION' is now connected ✅🔗
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF status' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'PREARM' is now connected ✅🔗
Vehicle 1: Action 'PREARM' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'MODE: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'MODE: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'ARM' is now connected ✅🔗
Vehicle 1: Action 'ARM' is now connected ✅🔗
Vehicle 1: Step 'wait' is now connected ✅🔗
Vehicle 1: Action 'WAIT' is now connected ✅🔗
Vehicle 1: Action 'WAIT' is now connected ✅🔗
Vehicle 1: Step 'start_mission' is now con

In [6]:
plan

✅ <Plan '📋 '>
  ✅ <Action '📤 UPLOAD_MISSION'>
    ✅ <Step '🔹 clear uav missions'>
    ✅ <Step '🔹 load mission simple_mission'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF status'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️ MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  ✅ <Action '⏳ WAIT'>
    ✅ <Step '🔹 wait'>
  ✅ <Action '🚀 START_MISSION'>
    ✅ <Step '🔹 start_mission'>
  ✅ <Action '📤 UPLOAD_MISSION'>
    ✅ <Step '🔹 monitoring'>